# Monitoring Stack Quick Dashboard
Live metrics from the local Prometheus instance.

In [ ]:
# Install deps if not already present
# !pip install prometheus-api-client pandas matplotlib seaborn

In [ ]:
from prometheus_api_client import PrometheusConnect
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
sns.set_theme(style="whitegrid")

prom = PrometheusConnect(url="http://localhost:9090", disable_ssl=True)

## 1. Endpoint Uptime (Blackbox probe_success)

In [ ]:
uptime = prom.get_current_metric_value('probe_success')
uptime_df = pd.DataFrame(uptime)
uptime_df['value'] = uptime_df['value'].astype(int)

sns.barplot(x='instance', y='value', data=uptime_df)
plt.title('Endpoint Up/Down (1 = Up)')
plt.xticks(rotation=45, ha='right')
plt.show()

## 2. Probe Duration (Latency)

In [ ]:
latency = prom.get_current_metric_value('probe_duration_seconds')
latency_df = pd.DataFrame(latency)
latency_df['value'] = latency_df['value'].astype(float)

sns.barplot(x='instance', y='value', data=latency_df)
plt.title('HTTP Latency (seconds)')
plt.xticks(rotation=45, ha='right')
plt.show()

## 3. Redis Keys (example)

In [ ]:
keys = prom.get_current_metric_value('redis_db_keys')
if keys:
    keys_df = pd.DataFrame(keys)
    keys_df['value'] = keys_df['value'].astype(int)
    sns.barplot(x='instance', y='value', data=keys_df)
    plt.title('Redis Keys per DB')
    plt.show()
else:
    print("No redis keys metric found (yet).")

## 4. Node CPU Utilization

In [ ]:
cpu = prom.get_current_metric_value('(1 - avg(rate(node_cpu_seconds_total{mode="idle"}[1m])) by (instance)) * 100')
cpu_df = pd.DataFrame(cpu)
cpu_df['value'] = cpu_df['value'].astype(float)

sns.barplot(x='instance', y='value', data=cpu_df)
plt.title('CPU Utilization (%)')
plt.show()

## Extras
- Adjust Prometheus queries to fit your rules
- Export this notebook (`File → Export → HTML`) and host the static HTML as a GitHub Pages report